## Assignment 1

## HAN YAOWEI (6930-31-2659)

### 0. Prepare 10 documents

In [1]:
doc = [''] * 10
doc[0] = 'How does the Surface Pro himself 4 compare with iPad Pro?'
doc[1] = 'Why did Microsoft choose core m3 and not core i3 home Surface Pro 4?'
doc[2] = 'Should I have a hair transplant at age 24? How much would it cost?'
doc[3] = 'How much cost does hair transplant require?'
doc[4] = 'What but is the best way to send money from China to the US?'
doc[5] = 'What you send money to China?'
doc[6] = 'Which food not emulsifiers?'
doc[7] = 'What foods fibre?'
doc[8] = 'How "aberystwyth" start reading?'
doc[9] = 'How their can I start reading?'

### 1. Set based similarity (Only report the results which not equal 0)

#### (0) Preprocessing

In [2]:
import string

def cleaning_text(text):
    remove = str.maketrans('','',string.punctuation) 
    text = text.translate(remove)
    return text

In [3]:
from nltk import word_tokenize

def tokenize_text(text):
    tokens = word_tokenize(text)
    return tokens

In [4]:
from nltk.corpus import wordnet as wn # import lemmatize

def lemmatize_word(word):
    word=word.lower()
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma

In [5]:
import nltk

en_stop = nltk.corpus.stopwords.words('english')
def remove_stopwords(word, stopwordset):
    if word in stopwordset:
        return None
    else:
        return word

In [6]:
set_doc = dict()

def preprocessing_text(text):
    text = cleaning_text(text)
    tokens = tokenize_text(text)
    tokens = [lemmatize_word(word) for word in tokens]
    tokens = [remove_stopwords(word, en_stop) for word in tokens]
    tokens = [word for word in tokens if word is not None]
    return tokens

preprocessed_docs = [preprocessing_text(text) for text in doc]
for i in range(10):
    set_doc[i] = set(preprocessed_docs[i])
print(set_doc)

{0: {'doe', 'surface', 'ipad', 'compare', '4', 'pro'}, 1: {'m3', 'surface', 'microsoft', 'home', 'core', '4', 'i3', 'pro', 'choose'}, 2: {'cost', 'would', 'age', 'transplant', '24', 'much', 'hair'}, 3: {'doe', 'cost', 'require', 'transplant', 'much', 'hair'}, 4: {'china', 'us', 'way', 'money', 'best', 'send'}, 5: {'money', 'china', 'send'}, 6: {'food', 'emulsifier'}, 7: {'food', 'fibre'}, 8: {'aberystwyth', 'reading', 'start'}, 9: {'reading', 'start'}}


#### (1) Jaccard index

In [7]:
from nltk.metrics import jaccard_distance

for i in range(10):
    for j in range(i + 1, 10):
        if 1 - jaccard_distance(set_doc[i], set_doc[j]) != 0:
            print('The jaccard_similarity between doc{0} and doc{1} is {2}'.format(i, j, 1 - jaccard_distance(set_doc[i], set_doc[j])))

The jaccard_similarity between doc0 and doc1 is 0.25
The jaccard_similarity between doc0 and doc3 is 0.09090909090909094
The jaccard_similarity between doc2 and doc3 is 0.4444444444444444
The jaccard_similarity between doc4 and doc5 is 0.5
The jaccard_similarity between doc6 and doc7 is 0.33333333333333337
The jaccard_similarity between doc8 and doc9 is 0.6666666666666667


We can see that since jaccard_similarity defines the similarity and diversity of two sets, the higher the proportion of intersections, the more similar the two documents.

However, if one set's size is too large, the value is small, no matter how large the intersection.

#### (2) Sørensen–Dice coefficient

In [8]:
def dice_similarity(set_a, set_b):
    num_intersection =  len(set.intersection(set_a, set_b))
    sum_nums = len(set_a) + len(set_b)
    try:
        return 2 * num_intersection / sum_nums
    except ZeroDivisionError:
        return 1.0

In [9]:
for i in range(10):
    for j in range(i + 1, 10):
        if dice_similarity(set_doc[i], set_doc[j]) != 0:
            print('The dice_similarity between doc{0} and doc{1} is {2}'.format(i, j, dice_similarity(set_doc[i], set_doc[j])))

The dice_similarity between doc0 and doc1 is 0.4
The dice_similarity between doc0 and doc3 is 0.16666666666666666
The dice_similarity between doc2 and doc3 is 0.6153846153846154
The dice_similarity between doc4 and doc5 is 0.6666666666666666
The dice_similarity between doc6 and doc7 is 0.5
The dice_similarity between doc8 and doc9 is 0.8


To improve the Jaccard Index's above problem, Sørensen–Dice coefficient sets the denominator as the average value of the two sets.
Thus, we can see that dice_similarity seems more accurate.

#### (3) Overlap coefficient (Szymkiewicz–Simpson coefficient)

In [10]:
def simpson_similarity(list_a, list_b):
    num_intersection = len(set.intersection(set(list_a), set(list_b)))
    min_num = min(len(set(list_a)), len(set(list_b)))
    try:
        return num_intersection / min_num
    except ZeroDivisionError:
        if num_intersection == 0:
            return 1.0
        else:
            return 0

In [11]:
for i in range(10):
    for j in range(i + 1, 10):
        if simpson_similarity(set_doc[i], set_doc[j]) != 0:
            print('The simpson_similarity between doc{0} and doc{1} is {2}'.format(i, j, simpson_similarity(set_doc[i], set_doc[j])))

The simpson_similarity between doc0 and doc1 is 0.5
The simpson_similarity between doc0 and doc3 is 0.16666666666666666
The simpson_similarity between doc2 and doc3 is 0.6666666666666666
The simpson_similarity between doc4 and doc5 is 1.0
The simpson_similarity between doc6 and doc7 is 0.5
The simpson_similarity between doc8 and doc9 is 1.0


The overlap coefficient is defined as the size of the intersection divided by the smaller of the size of the two sets.
Thus, we can see that if words in one document totally appear in another document, the overlap coefficient is 1.
It looks like it cannot reflect the similarity very well.

### 2. Vector based similarity (Only report the results which not equal 0)

#### (0) Transfer each document to vectorized representation using BoW. 

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(doc)
vector = vectorizer.transform(doc)
BoW = vector.toarray()
print(BoW)

[[0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 2 0
  0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 1 0 2 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]
 [1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0
  0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0
  1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0
  0 1 0 0 0 2 0 2 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
  0 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 

#### (1) Minkowski distance

##### (1).a Manhattan distance when p = 1

In [13]:
from scipy.spatial import distance

for i in range(10):
    for j in range(i + 1, 10):
        if distance.minkowski(BoW[i], BoW[j], 1) != 0:
            print('The manhattan_distance between doc{0} and doc{1} is {2}'.format(i, j, distance.minkowski(BoW[i], BoW[j], 1)))

The manhattan_distance between doc0 and doc1 is 19.0
The manhattan_distance between doc0 and doc2 is 20.0
The manhattan_distance between doc0 and doc3 is 13.0
The manhattan_distance between doc0 and doc4 is 22.0
The manhattan_distance between doc0 and doc5 is 16.0
The manhattan_distance between doc0 and doc6 is 14.0
The manhattan_distance between doc0 and doc7 is 13.0
The manhattan_distance between doc0 and doc8 is 12.0
The manhattan_distance between doc0 and doc9 is 13.0
The manhattan_distance between doc1 and doc2 is 25.0
The manhattan_distance between doc1 and doc3 is 20.0
The manhattan_distance between doc1 and doc4 is 27.0
The manhattan_distance between doc1 and doc5 is 19.0
The manhattan_distance between doc1 and doc6 is 15.0
The manhattan_distance between doc1 and doc7 is 16.0
The manhattan_distance between doc1 and doc8 is 17.0
The manhattan_distance between doc1 and doc9 is 18.0
The manhattan_distance between doc2 and doc3 is 9.0
The manhattan_distance between doc2 and doc4 is

Manhattan distance seems not a good metric since there seems no limit to the scale of distance, and we cannot directly get the result of how these two documents similar or not.

##### (1).b Euclidean distance when p = 2

In [14]:
for i in range(10):
    for j in range(i + 1, 10):
        if distance.minkowski(BoW[i], BoW[j], 2) != 0:
            print('The euclidean_distance between doc{0} and doc{1} is {2}'.format(i, j, distance.minkowski(BoW[i], BoW[j], 2)))

The euclidean_distance between doc0 and doc1 is 4.58257569495584
The euclidean_distance between doc0 and doc2 is 4.69041575982343
The euclidean_distance between doc0 and doc3 is 3.872983346207417
The euclidean_distance between doc0 and doc4 is 5.0990195135927845
The euclidean_distance between doc0 and doc5 is 4.242640687119285
The euclidean_distance between doc0 and doc6 is 4.0
The euclidean_distance between doc0 and doc7 is 3.872983346207417
The euclidean_distance between doc0 and doc8 is 3.7416573867739413
The euclidean_distance between doc0 and doc9 is 3.872983346207417
The euclidean_distance between doc1 and doc2 is 5.196152422706632
The euclidean_distance between doc1 and doc3 is 4.69041575982343
The euclidean_distance between doc1 and doc4 is 5.744562646538029
The euclidean_distance between doc1 and doc5 is 4.58257569495584
The euclidean_distance between doc1 and doc6 is 4.123105625617661
The euclidean_distance between doc1 and doc7 is 4.242640687119285
The euclidean_distance bet

Similar to manhattan distance, there seems no limit to the scale of distance, and we cannot directly get the result of how these two documents similar or not.

#### (2) Cosine similarity

In [15]:
import numpy as np

def cosine_similarity(x, y, norm=False):
    assert len(x) == len(y), "len(x) != len(y)"
    res = np.array([[x[i] * y[i], x[i] * x[i], y[i] * y[i]] for i in range(len(x))])
    cos = sum(res[:, 0]) / (np.sqrt(sum(res[:, 1])) * np.sqrt(sum(res[:, 2])))

    return 0.5 * cos + 0.5 if norm else cos

In [16]:
for i in range(10):
    for j in range(i + 1, 10):
        if cosine_similarity(BoW[i], BoW[j]) != 0:
            print('The cos_similarity between doc{0} and doc{1} is {2}'.format(i, j, cosine_similarity(BoW[i], BoW[j])))

The cos_similarity between doc0 and doc1 is 0.223606797749979
The cos_similarity between doc0 and doc2 is 0.08333333333333334
The cos_similarity between doc0 and doc3 is 0.2182178902359924
The cos_similarity between doc0 and doc4 is 0.13608276348795434
The cos_similarity between doc0 and doc8 is 0.14433756729740646
The cos_similarity between doc0 and doc9 is 0.12909944487358055
The cos_similarity between doc1 and doc6 is 0.12909944487358055
The cos_similarity between doc2 and doc3 is 0.545544725589981
The cos_similarity between doc2 and doc8 is 0.14433756729740646
The cos_similarity between doc2 and doc9 is 0.12909944487358055
The cos_similarity between doc3 and doc8 is 0.1889822365046136
The cos_similarity between doc3 and doc9 is 0.16903085094570328
The cos_similarity between doc4 and doc5 is 0.5773502691896258
The cos_similarity between doc4 and doc7 is 0.13608276348795434
The cos_similarity between doc5 and doc7 is 0.23570226039551587
The cos_similarity between doc8 and doc9 is 0.6

Cosine similarity looks good; the similarity of two docs can be directly inferred. 

### 3. Compare set-based and vector-based similarity calculation methods

Set based similarity calculation is more efficient because the computation is done on the small piece of word sets.

If the size of the document is large enough, vector based methods are more useful. However, the calculation amount will increase as the corpus increases.